In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.0 MB/s 
     |████████████████████████████████| 880 kB 46.1 MB/s 
     |████████████████████████████████| 3.3 MB 47.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=4deb34e92d74e0674ec0cc7268cb02ea7b302b6327918159bdb61af7eed6462d
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=21c8c0815a576b0a9a7b4b1eef11497d62ff65a2e0cdb30bfee0d781868df770
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
Looking in indexes: https://pypi.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

In [6]:
import tensorflow as tf

from tensorflow.python import keras
import json
import pandas as pd
import numpy as np
import os

from src.encoders.context_encoder_ldabert_2 import ContextEncoderComplex
from src.dataset.ldabert_2 import LDABERT2Dataset
import transformers
import sentence_transformers

from utils.metrics import windowdiff, pk

In [ ]:
tf.__version__, transformers.__version__, sentence_transformers.__version__

('2.6.0', '4.3.3', '0.4.1')

## LDA BERT 2 Dataset

In [17]:
dataset_type = "clinical"
dataset = LDABERT2Dataset(dataset_type=dataset_type,
                           pct_data=0.075,
                          max_seq_length=128,
                           max_segment_length=300,
                           augment_pct=0)

sentences, tokenized_sentences, labels = dataset.process()

# vectors_path = '../data/clinical_vectors/lda_bert_{}_{}.pkl'.format(dataset_type, len(sentences))
vectors_filename = '{}_{}.pkl'.format(dataset.pct_data, dataset.augment_pct)

saved_vectors, saved_labels, saved_tokens, saved_tokenized_sentences = dataset.get_saved_vectors("test", dataset.dataset_type, vectors_filename)

if len(saved_vectors) == 0:
    saved_vectors, saved_labels, saved_tokens, saved_tokenized_sentences = dataset.create_vectors("test", dataset.dataset_type, vectors_filename)

artificial segments False
artificial segments False


In [18]:
left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)

In [19]:
lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

In [20]:
saved_tokenized_sentences['input_ids'].shape, saved_vectors.shape

(TensorShape([2285, 128]), (2285, 12))

In [21]:
saved_tokens

['The arterial pulse changes as it travels from the central aorta down to the peripheral arteries ',
 'Although mean blood pressure decreases from the central aorta to the peripheral arteries  the systolic pressure increases ',
 'The pulse can be distorted and damped by reflected  resonance  or standing waves  as well as differences in the elastic properties and caliber of the peripheral arteries of the upper and lower parts of the body ',
 'The size of the normal arterial pulse is affected by the left ventricular ejection volume and the rate of ejection  as well as the elasticity or distensibility of the peripheral arteries ',
 'The greater the distance from the heart to a peripheral artery  the greater will be the distortion of the arterial pulse peripherally ',
 'Because complex factors affect the contour of the arterial pulse as it is transmitted through the arterial system  careful examination of the most central arterial pulse  the carotid  is necessary for pulse contour informat

## Model

In [35]:
lda_gamma = 2
model = ContextEncoderComplex(final_dropout=0.25,
                            dense_neurons=256,
                                  lstm_size=256,
                             lstm_dropout_percentage=0.25,
                             cnn_filters=8,
                             cnn_kernel_size=3,
                             pool_size=2,
                          gamma=lda_gamma,
                          max_sentence_length=128,
                           bert_trainable=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [36]:
hash = "hboPk"
model_path = 'LDABERT2/hybrid-conv/wiki-40561-1-pct-1-aug_{}/finetune-True'.format(hash)
checkpoint_filepath = config.root_path+'/models/{}/checkpoint'.format(model_path)
log_file = '{}/predictions/{}/predictions.csv'.format(config.root_path, hash)

In [37]:
# try:
model.load_weights(checkpoint_filepath)
# except:
#     print("No checkpoint available.")

In [38]:
saved_tokenized_sentences

{'input_ids': <tf.Tensor: shape=(2285, 128), dtype=int32, numpy=
array([[  101,  1996, 25543, ...,     0,     0,     0],
       [  101,  2348,  2812, ...,     0,     0,     0],
       [  101,  1996,  8187, ...,     0,     0,     0],
       ...,
       [  101, 21961, 23586, ...,     0,     0,     0],
       [  101,  2035,  1997, ...,     0,     0,     0],
       [  101,  1996,  3019, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2285, 128), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2285, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=in

## Inference

In [39]:
slice_size = 20
predictions_log = []

# for threshold in [0.25, 0.5, 0.75, 0.8]:
for threshold in [0.9]:
  for i in range(0, saved_vectors.shape[0]//slice_size):
      start = i*slice_size
      end = start + slice_size
      print(start, end)

      sliced_tokenized_sentences = {
          'input_ids': saved_tokenized_sentences['input_ids'][start:end],
          'token_type_ids': saved_tokenized_sentences['token_type_ids'][start:end],
          'attention_mask': saved_tokenized_sentences['attention_mask'][start:end]
      }
      sliced_vectors = saved_vectors[start:end]
      
      left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(sliced_tokenized_sentences)
      lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(sliced_vectors)
      
      logits = model([
          left_input, 
          mid_input, 
          right_input, 
          lda_left_input, 
          lda_mid_input, 
          lda_right_input
      ])

      logits_flattened = [float(p) for p in logits]

      pred_threshold = 0.5
      predictions = [1 if float(logit) >= threshold else 0 for logit in logits_flattened]

      k = 14

      string_predictions = "".join([str(i) for i in predictions])
      string_ground_truth = "".join([str(i) for i in saved_labels[start:end]])
      print(string_predictions)
      print(string_ground_truth)
      overall_windowdiff = windowdiff(string_predictions, string_ground_truth, k)
      overall_pk = pk(string_predictions, string_ground_truth, k)

      for pred_idx, pred in enumerate(predictions):
        log = {
            "prediction": pred,
            "ground_truth": saved_labels[start:end][pred_idx],
            "text": saved_tokens[start:end][pred_idx],
            "lda_gamma": lda_gamma,
            "prediction_threshold": threshold,
            "logit": logits_flattened[pred_idx]
        }
        predictions_log.append(log)
        print(log)

      print("{},{},{},{}".format(overall_windowdiff, overall_pk, k, k))
      break

0 20
sbert_output TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor: shape=(20, 128, 768), dtype=float32, numpy=
array([[[-0.03910301, -0.04485929,  0.05542713, ..., -2.170116  ,
          1.3732884 , -1.4447622 ],
        [-0.0365675 , -0.04529231,  0.05603777, ..., -2.1703708 ,
          1.3714181 , -1.445884  ],
        [-0.03732204, -0.04535982,  0.05585343, ..., -2.1698048 ,
          1.3706298 , -1.4442697 ],
        ...,
        [-0.03687917, -0.04479541,  0.05583953, ..., -2.1688712 ,
          1.3709574 , -1.4458683 ],
        [-0.03683222, -0.04480467,  0.05575506, ..., -2.1688652 ,
          1.370913  , -1.4458514 ],
        [-0.03682491, -0.0448495 ,  0.05575714, ..., -2.1688645 ,
          1.3708237 , -1.4459021 ]],

       [[-0.0392703 , -0.04243673,  0.05797173, ..., -2.1698296 ,
          1.3714203 , -1.4466132 ],
        [-0.03893703, -0.04241228,  0.05815948, ..., -2.1688385 ,
          1.3708626 , -1.4480797 ],
        [-0.03652827, -0.0420638 ,  0.0593344 , 

In [24]:
try:
  import csv
  if not os.path.exists(os.path.dirname(log_file)):
      os.makedirs(os.path.dirname(log_file))
  mode = 'a' if os.path.exists(log_file) else 'w'
  with open(log_file, mode) as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=["prediction", "ground_truth", "text", "lda_gamma", "prediction_threshold", "logit"])
      writer.writeheader()
      for data in predictions_log:
          writer.writerow(data)
except IOError:
    print("I/O error")

In [ ]:
[float(p) for p in predictions], saved_labels

([0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,